In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from panoradio_hf.data import IQDataModel
from panoradio_hf.ccnn import ClassicalCNN
from panoradio_hf.net_utils import initialize_pl_trainer
import lightning as pl

In [5]:
iqdata = IQDataModel()
model = ClassicalCNN()

callbacks, mlf_logger = initialize_pl_trainer("classical-cnn")

trainer = pl.Trainer(callbacks=callbacks,
                     max_epochs=100,
                     logger=mlf_logger)

trainer.fit(model, iqdata)